In [1]:
import cv2
import numpy as np
import pandas as pd

df_to_plot = pd.read_pickle('/content/drive/MyDrive/Project/Object_tracking/scan1_joints_and_masks_to_plot_updated.pkl')

In [2]:
def concatenate_lists(list_of_lists):
    result = []
    for sublist in list_of_lists:
        result += sublist
    return result

def get_points_to_plot(frame_id, threshold_dis = 0.03):
  inters = []
  objs = []
  alphas = []
  is_near = concatenate_lists(df_to_plot[df_to_plot['frame_id'] == frame_id]['is_near'].values[0])
  inter_masks = concatenate_lists(df_to_plot[df_to_plot['frame_id']==frame_id]['interactions'].values[0])
  obj_masks = concatenate_lists(df_to_plot[df_to_plot['frame_id']==frame_id]['masks'].values[0])
  for n in range(len(inter_masks)):
    if is_near[n]:
      obj_xys = obj_masks[n][:, :2]
      objs.append(obj_xys)
      if not all(v is None for v in inter_masks[n]):
        inter_points = [i for i in inter_masks[n] if i is not None]
        inter_points = np.concatenate(inter_points)
        inter_xys = inter_points[:, :2]
        inter_alpha = inter_points[:, -1:] / threshold_dis
        inters.append(inter_xys)
        alphas.append(inter_alpha)
  return objs, inters, alphas

In [3]:
def plot_mask_points(frame, coordinates, color=(0, 0, 255), alpha=0.7):
    overlay = frame.copy()
    for point in coordinates:
        cv2.circle(overlay, (int(point[0] * frame.shape[1]), int(point[1] * frame.shape[0])), 5, color, -1)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)


def plot_points_with_alpha(frame, points, alphas, color=(0, 165, 255)):
    for point, alpha in zip(points, alphas):
        overlay = frame.copy()
        cv2.circle(overlay, (int(point[0] * frame.shape[1]), int(point[1] * frame.shape[0])), 5, color, -1)
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

In [4]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Project/Object_tracking/scan_video1.avi')

# Initialize video writer
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter('/content/drive/MyDrive/Project/Object_tracking/scan_video1_with_masks.avi', cv2.VideoWriter_fourcc('X', 'V', 'I', 'D'), cap.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))

frame_range = range(0, length)
key_frames = df_to_plot['frame_id'].tolist()

current_frame = frame_range[0]
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = frame.copy()
    if current_frame in frame_range and current_frame in key_frames:
        objs, inters, alphas = get_points_to_plot(current_frame)

        for obj_coordinates in objs:
            plot_mask_points(processed_frame, obj_coordinates, color=(0, 0, 255), alpha=0.6)  # Red color for regular masks

        for inter_points, inter_alphas in zip(inters, alphas):
            plot_points_with_alpha(processed_frame, inter_points, inter_alphas.flatten(), color=(0, 165, 255))  # Light orange for alpha masks

    out.write(processed_frame)  # Write every frame
    current_frame += 1
    if current_frame > frame_range[-1]:
        break

cap.release()
out.release()